In [ ]:
%run -i ../python/common.py
UC_SKIPTERMS=True
%run -i ../python/ln_preamble.py

# SLS Lecture 14 : Assembly using the OS

- Review of Processes
- Review of Systems Calls
- Process Address Spaces 
  - Dynamic Memory : Growing our memory
- I/O - Assembly "Hello World"

In [ ]:
# setup for sumit examples
appdir=os.getenv('HOME')
appdir=appdir + "/syscalls"
#print(movdir)
output=runTermCmd("[[ -d " + appdir + " ]] &&  rm -rf "+ appdir + 
             ";mkdir " + appdir + 
             ";cp ../src/Makefile ../src/setup.gdb ../src/usesum5.S ../src/sumit6.S ../src/hello.S ../src/brk.S " + appdir)

display(Markdown('''
- create a directory `mkdir syscalls; cd syscalls`
- copy examples
- add a `Makefile` to automate assembling and linking
    - we are going run the commands by hand this time to highlight the details
- add our `setup.gdb` to make working in gdb easier
- normally you would want to track everything in git
'''))
TermShellCmd("ls " + appdir)

## Review of Processes

### Processes and the Kernel
<img src="../images/UnixRunning.png">

In [ ]:
display(HTML(htmlFig(
    [
        [
#            {'src':"/files/work/UndertheCovers/underthecovers/images/Processes/Processes.003.png",
#             'caption':'A: Press Enter', 
#             'border': '1px solid black',
#             'padding':'1px',
#             'cellwidth':'33.33%'
#            },
            {'src':"../images/Processes/Processes.004.png",
#             'caption':'B: Shell "blank line" input processing' , 
#             'border': '1px solid black',
#             'padding':'1px',
#             'cellwidth':'33.33%'
            },
            {'src':"../images/Processes/Processes.005.png",
#             'caption':'C: Shell sends Prompt back', 
#             'border': '1px solid black',
#             'padding':'1px',
#             'cellwidth':'33.33%'
            },
        ]
    ],
#    id="fig:shell-blankline",
#    caption="<center> Figure: Shell blank line behavior </center>"
)))

## Review of System Calls - From Lecture 08

In [ ]:
display(Markdown(htmlFig("../images/kerneldef.png")))

Remember this...

**The "Kernel" -- Unique to Every OS**

1. Bootstraps the HW and has direct access to all of it
1. Bottom layer that enables other programs to run
2. <font style="color:red;"> **A unique collection of functions that programs can invoke** </font>

Not useful on its own only useful and accessed by running other programs.

<center>
<img src="../images/Processes/Processes.051.png">
</center>

### Intel `syscall`

On Intel the instruction is `syscall`

<img src="../images/syscallmp.png">



### The OS System Calls

Each OS Kernel provides a set of calls that an a process can invoke using the `syscall` instruction on an Intel based computer

The Linux Kernel supports a very large number of system calls each is identified by a unique number that must be placed in `RAX` prior to executing the `syscall` instruction.  Additional arguments are passed in by setting other registers.  

With each version of the Kernel the table of calls changes.  Here is one site that provides a list

### LINUX SYSTEM CALLS

- reading some man pages `man syscall` and `man syscalls` we find that
  - we must place `60` in `rax`
  - and that the value we want to return in `rdi`

In [ ]:
display(IFrame("https://filippo.io/linux-syscall-table/", height=600, width="100%"))

In [ ]:
display(Markdown(FileCodeBox(
    file="../src/exit_bb_bb.S", 
    lang="gas", 
    title="",
    h="100%", 
    w="100%"
)))

## Process Address Spaces

### Address Space organizes Memory of a Process

<center>
    <img src="../images/ProcessContexts.png">
</center>

**Remember**

1. The binaries we create using the assembler and linker are Executables

2. When we "run" our executables via the shell it creates a new process context and our binary is loaded as the initial memory "image".

3. The memory and register values of the process are unique to each process and change as the instructions of our binary "execute"

### Assembly Programming as Process Address Space Image creation

<center>
    <img src="../images/ASSEMBLY-AddressSpaceandIO/ASSEMBLY-AddressSpaceandIO.003.png">
</center>

<center>
    <img src="../images/ASSEMBLY-AddressSpaceandIO/ASSEMBLY-AddressSpaceandIO.004.png">
</center>

<center>
    <img src="../images/ASSEMBLY-AddressSpaceandIO/ASSEMBLY-AddressSpaceandIO.005.png">
</center>

### The Details

### Process Address Space 

The Memory of a Process is organized as an (Virtual) Address Space 

1. Each possible location of a byte is identified by a unique address (number) 
2. To associate a Region (range of continues addresses) with actual memory requires a "mapping"
    - mappings associate a Region with Memory and a possible source of values
    - mappings can restrict what kind of access can be made to region 
        1. fetch for execute (x)
        2. read (r)
        3. write (w)

<center>
    <img src="../images/ExecAddrSpace/ExecAddrSpace-001.png">
</center>

<center>
    <img src="../images/ExecAddrSpace/ExecAddrSpace-002.png">
</center>

<center>
    <img src="../images/ExecAddrSpace/ExecAddrSpace-003.png">
</center>

<center>
    <img src="../images/ExecAddrSpace/ExecAddrSpace-004.png">
</center>

<center>
    <img src="../images/ExecAddrSpace/ExecAddrSpace-005.png">
</center>

<center>
    <img src="../images/ExecAddrSpace/ExecAddrSpace-006.png">
</center>

<center>
    <img src="../images/ExecAddrSpace/ExecAddrSpace-007.png">
</center>

<center>
    <img src="../images/ExecAddrSpace/ExecAddrSpace-008.png">
</center>

<center>
    <img src="../images/ExecAddrSpace/ExecAddrSpace-009.png">
</center>

<center>
    <img src="../images/ExecAddrSpace/ExecAddrSpace-010.png">
</center>

<center>
    <img src="../images/ExecAddrSpace/ExecAddrSpace-011.png">
</center>

### Exploring the Address Space

- Let's explore the address space of a running program 

In [ ]:
display(Markdown(FileCodeBox(
    file="../src/usesum5.S", 
    lang="gas", 
    title="<b>CODE: asm - usesum5.S",
    h="100%", 
    w="107em"
)))

In [ ]:
display(Markdown(FileCodeBox(
    file="../src/sumit6.S", 
    lang="gas", 
    title="<b>CODE: asm - sumit6.S",
    h="100%", 
    w="107em"
)))

In [ ]:
TermShellCmd("make usesum6", cwd=appdir, prompt='')

In [ ]:
showDT()

```
b _start
run 
info proc
```

In [ ]:
TermShellCmd("man proc | head -30", markdown=False)

In [ ]:
TermShellCmd("man proc | grep -A40 /maps | head -41", markdown=False)

```
 cat /proc/<pid>/maps
```

Use gdb to explore what's there and compare with binary binary file 

#### Add another section to control how region is mapped

Readonly data section `.rodata`

```gas
         .section .rodata

 str1:
         .string "Hello World\n"
```

What would happen if we put all out data in the `.rodata` section... try it and find out

## Ok lets get some more memory -- Grow our address space

The `brk` system call


In [ ]:
TermShellCmd("man brk | head -37", markdown=False)

In [ ]:
display(IFrame("https://filippo.io/linux-syscall-table/", height=600, width="100%"))

#### Lets add this code

What does it do?

```gas
    .data
NEW_MEM_PTR:
    .quad 0

    .text
    
    mov rdi, 0
    mov rax, 12
    syscall 
    
    mov QWORD PTR [NEW_MEM_PTR], rdi
    
    # look at RAX
    
    add rdi, 1024
    mov rax, 12
    syscall
    
    # look at RAX
```

#### What does the regions maps look like now?

<center>
    <img src="../images/ASSEMBLY-AddressSpaceandIO/ASSEMBLY-AddressSpaceandIO.012.png">
</center>

<center>
    <img src="../images/ASSEMBLY-AddressSpaceandIO/ASSEMBLY-AddressSpaceandIO.016.png">
</center>

## I/O

<center>
    <img src="../images/ASSEMBLY-AddressSpaceandIO/ASSEMBLY-AddressSpaceandIO.017.png">
</center>

<center>
    <img src="../images/ASSEMBLY-AddressSpaceandIO/ASSEMBLY-AddressSpaceandIO.018.png">
</center>

### Review from Lecture 3

<center>
    <img src="../images/Processes/Processes.006.png">
</center>

<center>
    <img src="../images/Processes/Processes.007.png">
</center>

<center>
    <img src="../images/Processes/Processes.008.png">
</center>

<center>
    <img src="../images/Processes/Processes.009.png">
</center>

<center>
    <img src="../images/Processes/Processes.010.png">
</center>

<center>
    <img src="../images/Processes/Processes.011.png">
</center>

<center>
    <img src="../images/Processes/Processes.012.png">
</center>

<center>
    <img src="../images/Processes/Processes.013.png">
</center>

<center>
    <img src="../images/Processes/Processes.015.png">
</center>

### Finally "Hello World in Assembly"

In [ ]:
display(Markdown(FileCodeBox(
    file="../src/hello.S", 
    lang="gas", 
    title="<b>CODE: asm - hello.S",
    h="100%", 
    w="107em"
)))

In [ ]:
TermShellCmd("make hello", cwd=appdir, prompt='')

#### At long last we have a program that we don't need the debugger for!

In [ ]:
TermShellCmd("./hello; echo $?", cwd=appdir)

### There are many more calls for doing I/O

But we can now 
1. transfer bytes from our address space to a "file" - write system call
2. transfer bytes from a "file" to our address space  - read system call
3. connect (and possibly create) files to our address space for reading and writing - open system call
4. disconnect a file from our address space - close system call


In [ ]:
display(IFrame("https://filippo.io/linux-syscall-table/", height=600, width="100%"))

### Exercises

Change the `usesum` program to:
- exit properly and returning the result
- allocate memory for the data
- read the data in from standard in
- open the binary file of numbers and read them in

Write a new program that uses the random instruction to create a data file
